In [2]:
from sqlalchemy import create_engine
import duckdb
import pandas as pd

source_engine = create_engine('sqlite:///data/basic_examples.db')
source_connection = source_engine.connect()

tables = ["Dragon", "Dish", "Scene"]
for table in tables:
    query = "SELECT * FROM " + table 
    data = pd.read_sql(query, source_connection)
    
    duck_connection = duckdb.connect('data/example_duck.db')
    
    duck_connection.execute("DROP TABLE IF EXISTS " + table)
    
    if table == "Dragon":
        duck_connection.execute("""CREATE TABLE dragon (
        name VARCHAR PRIMARY KEY,
        year INTEGER CHECK (year >= 2000),
        cute INTEGER);""")

    elif table == "Dish":
        duck_connection.execute("""CREATE TABLE dish (
        name VARCHAR PRIMARY KEY,
        type VARCHAR,
        cost INTEGER CHECK (cost >= 0));""")

    else:
        duck_connection.execute("""CREATE TABLE scene (
        id INTEGER PRIMARY KEY,
        biome VARCHAR NOT NULL,
        city VARCHAR NOT NULL,
        visitors INTEGER CHECK (visitors >= 0),
        created_at DATETIME DEFAULT current_date());""")
    # duck_connection.execute("CREATE TABLE IF NOT EXISTS " + table + " AS SELECT * FROM data LIMIT 0") 
   
    duck_connection.register('data', data) 
    duck_connection.execute("INSERT INTO " + table + " SELECT * FROM data") 

source_connection.close()
duck_connection.close()